# Загрузка данных

In [1]:
import zipfile
import pandas as pd

In [2]:
with zipfile.ZipFile('../datasets/client_info.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('../datasets')

In [ ]:
df = pd.read_csv('../datasets/client_info.csv')
df.head(5)

# 1
Назначаем  `ID` индексом таблицы.

In [ ]:
df = df.set_index('ID')
df.head(5)

## 2
- Сохраняем признаки в переменную X.
- Сохраняем целевую переменную в Y.

In [ ]:
y = df[['TARGET']] #  целевая переменная
#loc используется для индексации и выбора данных по меткам (лейблам) строк и столбцов.
x = df.loc[:, df.columns != 'TARGET'] # признаки
y.shape , x.shape 

## 3
Сохраняем категоральные и чмсловые признаки

In [ ]:
# Посомтрел какие типы данных входят

# Применил функцию select_dtypes (include = включить, exclude = исключить)
X_cat = x.select_dtypes(include=['object', 'category'])
X_num = x.select_dtypes(exclude=['object', 'category']) 

X_cat.shape [1], X_num.shape[1]

## 4
- Предобрабатываем количественные переменные. 
- Проведим стандартизацию для признаков X_num.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Создаем экземпляр класса StandardScaler для стандартизации данных
scaler = StandardScaler()

#делаем стандартизацию наших признаков
# Применяем метод fit_transform для стандартизации числовых признаков X_num
# fit_transform выполняет обучение на данных и преобразование данных
X_num_scal = scaler.fit_transform(X_num) 

# Преобразуем полученный массив X_num_scal в объект DataFrame с исходными названиями столбцов
X_num_scal = pd.DataFrame (X_num_scal, columns = X_num.columns)


X_num_scal_standart = round(X_num_scal.std(),2) 
X_num_scal_mean = round(X_num_scal.mean(), 2)

Standar_and_mean = pd.concat([X_num_scal_standart, X_num_scal_mean], axis= 1 )
Standar_and_mean.columns = ['Standart', 'Mean'] 
Standar_and_mean


## 5
Трансформируем категориальные признаки. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder 

# Удаляем столбец 'CLNT_JOB_POSITION' из DataFrame X_cat
X_cat = X_cat.drop(columns='CLNT_JOB_POSITION')

# Создаем экземпляр объекта OneHotEncoder и преобразуем X_cat в разреженную матрицу
encoder = OneHotEncoder()
X_cat_ohe = encoder.fit_transform(X_cat).toarray()

# Создаем DataFrame из массива X_cat_ohe с именами столбцов, полученными от OneHotEncoder
X_cat_ohe = pd.DataFrame(X_cat_ohe, columns=encoder.get_feature_names_out())

# Возвращаем количество столбцов в X_cat_ohe
X_cat_ohe.shape[1]

## 6
Разделяем данные на обучающий и тестовый наборы.


In [ ]:
from sklearn.model_selection import train_test_split

X_ready = pd.concat ([X_num_scal, X_cat_ohe],axis= 1 )
X_ready.to_csv('../datasets/data_prepared.csv')
X_train, X_test, y_train, y_test = train_test_split( X_ready, y, test_size=0.2, random_state=21)

# Разделяем данные на обучающий и тестовый наборы, где:
# X_ready - матрица признаков
# y - целевая переменная
# test_size=0.2 - указывает, что 20% данных будут использованы для тестирования
# random_state=21 - установка случайного состояния для воспроизводимости результатов

X_train.shape


## 7 
Обучаем логистическую регрессию 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Создаем экземпляр модели LogisticRegression с определенными параметрами
# solver='liblinear' - выбор метода решения задачи оптимизации
# fit_intercept=False - отключение подбора свободного члена (intercept)
# penalty='l1' - применение L1-регуляризации
# random_state=21 - установка случайного состояния для воспроизводимости результатов

# Обучаем модель на обучающих данных X_train и y_train
# X_train.values - преобразование данных X_train в формат массива значений
# y_train - целевая переменная для обучения модели

# Применяем обученную модель для предсказания значений целевой переменной на тестовых данных X_test
# X_test.values - преобразование данных X_test в формат массива значений

mod = LogisticRegression (solver='liblinear', 
                         fit_intercept=False, 
                         penalty='l1', 
                         random_state=21).fit(X_train.values, y_train)


# Вычисляем точность (accuracy) предсказания модели на тестовых данных
# Сравниваем предсказанные значения y_pred с истинными значениями y_test

y_pred = mod.predict(X_test.values)
target = accuracy_score(y_test, y_pred)
target

## 8
Считаем процент клиентов в тестовой выборке, у которых индикатор оттока равен 0.

In [ ]:
import numpy as np

percent_client = np.mean(y_test == 0) * 100
percent_client

## 9
Строим график, на котором видны топ-10 самых важных факторов по мнению модели в абсолютном значении.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# извлекает коэффициенты модели из атрибута coef_ и сохраняет их в переменную 
coef_model = mod.coef_[0]
# преобразует массив c в объект NumPy  
array_model = np.array(coef_model)
# вычисляет абсолютные значения всех элементов массива с помощью функции np.abs 
absolute = np.abs (array_model)

total = pd.Series(absolute, index = X_ready.columns)
total_sorted = total.sort_values(ascending=False)

sns.barplot(x=total_sorted.values[:10], y=total_sorted.index[:10], palette='Set2')
plt.title('Топ 10')
plt.xlabel('Значения коэфицентов')
plt.ylabel('Факторы')

plt.show()